1:16 
[Link](https://www.linkedin.com/learning/nlp-with-python-for-machine-learning-essential-training/implementation-removing-punctuation?u=74412268)

Steps to NLP for ML:

1) Raw text - mdoel cannot read text

2) Tokenizing - telling model what to read from file 

3) Clean text - remove stop words, punctuation, stemming, etc.

4) Vectorize - turn data to numeric form

5) Machine Learning - put into model for train/test
    - classification model

## 1) Preprocess Data

In [11]:
import pandas as pd
pd.set_option("display.max_colwidth", 100) #set layout table

my_data = pd.read_csv("Restaurant_Reviews.tsv", sep="\t", header=None)
my_data.columns = ["Review","Positive/Negative rating"]

my_data[-5:]

,Review,Positive/Negative rating
996,I think food should have flavor and texture and both were lacking.,0
997,Appetite instantly gone.,0
998,Overall I was not impressed and would not go back.,0
999,"The whole experience was underwhelming, and I think we'll just go to Ninja Sushi next time.",0
1000,"Then, as if I hadn't wasted enough of my life there, they poured salt in the wound by drawing ou...",0
